In [96]:
import numpy as np
import pandas as pd
import scipy as sci
from ase import Atoms
from ase.visualize import view
from ase.io import write
from ase.io import read
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms
from rdkit.Chem import Draw

In [98]:
#import VASP output files
df = pd.read_csv('thermo_out_gas.csv')




In [99]:
ev_kJ = 96.4869
h_ev = 4.135668E-15 #planks constant in ev*s
k_ev = 0.0000861733 #boltzmann in ev/K
R_ig = 8.3144 #kJ/mol/K
T = 533

#initial vector to build S H and ZPEs
S_vec = np.array([])
H_vec = np.array([])
ZPE_vec = np.array([])
H_rot_vec = np.array([])
H_trans_vec = np.array([])
S_rot_ig_vec = np.array([])
S_trans_vec = np.array([])
f_ims = np.array([])
   

In [103]:
for i in range(df.shape[0]):
    [N_A,k_B,h,kJmol_Ha] = [6.0221408e23,1.380649e-23,6.626070E-34,2625.5] #[ -- , m2 kg s-2 K-1, K,  # m2 kg / s]
    R_ig = 8.3144
    R = k_B*N_A 
    lp = df['path_f'][i]
    print(lp)
    #get number of atoms at DOF
    f = open(df['path_g'][i]+'/CONTCAR','r')
    data = list([])
    
    for lines in f:
        data.append(lines)
    atoms = data[6].split(' ')
    ele=''
    atoms=[i for i in atoms if i!=ele]
    atoms[-1] = atoms[-1][0:-1]
    n_HC = list([])
    [n_HC.append(int(x)) for x in atoms]
    #here, no adsorbate so sum over all n_HC
    n_atoms = sum(n_HC)
    
    #moments
    raw_mol = Chem.MolFromXYZFile(df['path_g'][i]+'/out.xyz')
    mol = Chem.Mol(raw_mol)
    conf = mol.GetConformer()
    moments_of_inertia = rdMolTransforms.ComputePrincipalAxesAndMoments(conf)
    inertia_values = moments_of_inertia[1]*1.67377E-27/1e10/1e10
    print(inertia_values)
    
    
    
    if df['species'][i]=='H2':
        DoF = n_atoms*3-5
    else:
        DoF = n_atoms*3-6
    
    m = df['mass'][i]*1.67377E-27
    
    try:
        df_freq = pd.read_csv(lp+'/Freqs.txt',header=None,delim_whitespace=True)

    except ValueError:
        df_freq = np.array([])
    
    if np.size(df_freq)!= 0:
        df_freq = pd.read_csv(lp+'/Freqs.txt',header=None,delim_whitespace=True)
        vibs = df_freq[7]
        vibs = pd.to_numeric(vibs,errors='coerce')

        v_hz = vibs.to_numpy()*29979.2458*1e6#cm-1 -> Mhz -> hz
        S_vibs =R_ig/1000*((h_ev*v_hz/(k_ev*T*(np.exp(h_ev*v_hz/k_ev/T)-1))-np.log(1-np.exp(-h_ev*v_hz/k_ev/T))))
        ZPE = 1/2*h_ev*v_hz
        H_vib = h_ev*v_hz*np.exp(-h_ev*v_hz/k_ev/T)/(1-np.exp(-h_ev*v_hz/k_ev/T))
        if df['species'][i]=='Toluene': #remove the mode for internal rotation s
            S_vec = np.append(S_vec,np.sum(np.sum(S_vibs[0:(DoF-1)]))) 
            #Data['H_vib'][i] = np.sum(np.sum(S_damp))
            H_vec = np.append(H_vec,np.sum(np.sum(H_vib[0:(DoF-1)]))) 
        else:
            S_vec = np.append(S_vec,np.sum(np.sum(S_vibs[0:(DoF)]))) 
            #Data['H_vib'][i] = np.sum(np.sum(S_damp))
            H_vec = np.append(H_vec,np.sum(np.sum(H_vib[0:(DoF)])))         
        
        ZPE_vec = np.append(ZPE_vec,np.sum(np.sum(ZPE[0:DoF]))) 

        
    else:
        S_vec = np.append(S_vec,0) 
        H_vec = np.append(H_vec,0) 
        ZPE_vec = np.append(ZPE_vec,0) 
        
    #https://cccbdb.nist.gov/thermox.asp
    if df['species'][i]=='H2': #linear rotor
        sigma = 2
        H_rot = 3/2*R_ig*T/1000 # kJ/mol 
        B = h/8/np.pi/np.pi/4.64E-48
        S_rot = R_ig/1000*(np.log(k_B*T/sigma/h/B)+1)
    else: #non-linear rotor
        sigma = 1
        A = h/8/np.pi/np.pi/inertia_values[0]
        B = h/8/np.pi/np.pi/inertia_values[1]
        C = h/8/np.pi/np.pi/inertia_values[2]
        H_rot = 3/2*R_ig*T/1000 # kJ/mol    
        S_rot = R_ig/1000*(3/2*np.log(k_B*T/h)-1/2*np.log(A*B*C/np.pi)-np.log(sigma)+3/2)
        
        

    H_trans = 5/2*R_ig*T/1000#df_H.to_numpy()[0] # kJ/mol#populate the table
    S_trans = R_ig/1000*(3/2*np.log(2*np.pi*m/h/h)+5/2*np.log(k_B*T)-np.log(1E5)+5/2) # J/mol/K      
 

    H_rot_vec = np.append(H_rot_vec,H_rot)
    H_trans_vec = np.append(H_trans_vec,H_trans)
    S_rot_ig_vec = np.append(S_rot_ig_vec,S_rot)
    S_trans_vec = np.append(S_trans_vec,S_trans)
        
df.insert(2,'S_vib',S_vec) #kJ/mol
df.insert(2,'H_vib',H_vec*ev_kJ) #kJ/mol 
df.insert(2,'ZPE',ZPE_vec*ev_kJ) #kJ/mol

df.insert(2,'H_rot',H_rot_vec) #kcal/mol -> kJ/mol (occured at import)
df.insert(2,'H_trans',H_trans_vec) #kcal/mol -> kJ/mol
df.insert(2,'S_rot',S_rot_ig_vec) #kcal/mol -> J/mol/K
df.insert(2,'S_trans',S_trans_vec) #kcal/mol -> J/mol/K


/Users/arifischer/Documents/misc/MS2/DFT_calculations/compiling_calculations/gas/H2/freq
[0. 0. 0.]
/Users/arifischer/Documents/misc/MS2/DFT_calculations/compiling_calculations/gas/toluene/freq
[9.73916335e-47 2.22226108e-46 3.19607668e-46]
/Users/arifischer/Documents/misc/MS2/DFT_calculations/compiling_calculations/gas/4MCHE/freq
[1.09094905e-46 2.33780195e-46 3.34364061e-46]


In [104]:
#calculate get the internal rotation for toluene
raw_mol = Chem.MolFromXYZFile(df['path_g'][1]+'/out.xyz')
mol = Chem.Mol(raw_mol)
conf = mol.GetConformer()
moments_of_inertia = rdMolTransforms.ComputePrincipalAxesAndMoments(conf)

In [105]:
### get the axis for the C-C bond with the methyl group
#methyl group, C7
atom_idx1 = 5  # Example: Atom 1
atom_idx2 = 6  # Example: Atom 2
# Get the coordinates of the two atoms
coords1 = np.array(conf.GetAtomPosition(atom_idx1))
coords2 = np.array(conf.GetAtomPosition(atom_idx2))
# Calculate the bond vector
bond_vector = coords2 - coords1
# Normalize the bond vector to get the unit vector (axis along the bond)
bond_axis = bond_vector / np.linalg.norm(bond_vector)
bond_axis

array([ 0.16006344,  0.98704949, -0.01063034])

In [106]:
#get alpha, beta, kappa, the cosine of the angles with the axis
cos_thetas = list([])
for i in moments_of_inertia[0]:
    # Calculate the dot product of the two vectors
    dot_product = np.dot(i, bond_axis)

    # Calculate the magnitudes of the vectors
    magnitude_axis1 = np.linalg.norm(i)
    magnitude_axis2 = np.linalg.norm(bond_axis)

    # Calculate the cosine of the angle
    cos_theta = dot_product / (magnitude_axis1 * magnitude_axis2)
    print(cos_theta)
    cos_thetas.append(cos_theta)
#calc moment part
np.array(cos_thetas)**2/moments_of_inertia[1]

0.9991645054059124
0.0007487966403851348
0.04086233523123881


array([1.71572676e-01, 4.22307187e-08, 8.74429809e-05])

In [107]:
#find the distances of the H to the axis
#methyl H at H6 H7 H8, last three atoms
H_list = np.array([conf.GetAtomPosition(12),conf.GetAtomPosition(13),conf.GetAtomPosition(14)])
# Define a point A on the axis
A = coords1

# Define the direction vector d of the axis
d = bond_axis  # Example: y-axis

ds = list([])

for i in H_list:

    P = i
    # Step 1: Calculate the vector AP
    AP = P - A

    # Step 2: Compute the cross product AP x d
    cross_product = np.cross(AP, d)

    # Step 3: Calculate the magnitude of the cross product
    magnitude_cross_product = np.linalg.norm(cross_product)

    # Step 4: Calculate the magnitude of the direction vector d
    magnitude_d = np.linalg.norm(d)

    # Step 5: Calculate the distance from point P to the axis
    distance = magnitude_cross_product / magnitude_d
    ds.append(distance)
ds#angstrom

[1.0293014724163907, 1.0228345211952214, 1.0227959124172359]

In [108]:
## find the moment of inertial for the internal rotation

#sum over mass * r**2, in units of kg and angstroms2 to m2
#I_top = np.sum((np.array(ds)**2)*1.67377E-27)/1E10/1E10
I_top = np.sum(np.array(ds)**2)
I_int = I_top-I_top**2*np.sum(np.array(cos_thetas)**2/moments_of_inertia[1])
I_int

1.4465578689042633

In [109]:
# calculate the entropy for the internal rotation

S_int = (R_ig/1000*
    (1/2*np.log(8*np.pi**3*I_int*1.67377E-27/1E10/1E10*k_B*533)-np.log(6*h)+1/2))
print(S_int*1000)
print(1/2*R_ig*T/1000)

8.43187772800907
2.2157875999999996


In [110]:
df.to_csv("thermo_gas_533_aug.csv")

In [93]:
df

,Unnamed: 0,species,S_trans,S_rot,H_trans,H_rot,ZPE,H_vib,S_vib,n_H,path_g,path_f,E_ev,mass
0,0,H2,0.129673,0.017641,11.078938,6.647363,25.962097,0.000424,8.624857e-07,0,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,-7.025669,2
1,1,Toluene,0.177423,0.086199,11.078938,6.647363,327.518652,31.986495,9.397899e-02,0,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,-92.846885,92
2,2,4MCHE,0.177953,0.087069,11.078938,6.647363,443.016440,44.395236,1.385537e-01,0,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,/Users/arifischer/Documents/misc/MS2/DFT_calcu...,-107.908470,96


In [90]:
S_rot_ig_vec

array([0.0176412 , 0.08619893, 0.08706903])

In [22]:
for atom in mol.GetAtoms():
    atom_index = atom.GetIdx()
    coords = conf.GetAtomPosition(atom_index)
    print(f"Atom Index: {atom_index}, Symbol: {atom.GetSymbol()}, Coordinates: {coords}")

Atom Index: 0, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5078c0>
Atom Index: 1, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f507940>
Atom Index: 2, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5078c0>
Atom Index: 3, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f507940>
Atom Index: 4, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5078c0>
Atom Index: 5, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5074c0>
Atom Index: 6, Symbol: C, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5078c0>
Atom Index: 7, Symbol: H, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5074c0>
Atom Index: 8, Symbol: H, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f5078c0>
Atom Index: 9, Symbol: H, Coordinates: <rdkit.Geometry.rdGeometry.Point3D object at 0x14f507940>
Atom Index: 10, Symbol: H, Coo

In [19]:
img = Draw.MolToImage(mol, size=(300, 300), kekulize=True, wedgeBonds=True, atomIdx=True)
img.show()

In [ ]:
#free energies:

#G = H - T S
#H + E + ZPE + H_vib
Hs = (df['E_ev'].to_numpy()*ev_kJ + df['ZPE'].to_numpy() + df['H_vib'].to_numpy()+
      df['H_rot'].to_numpy()+df['H_trans'].to_numpy())

Ss = df['S_vib'].to_numpy()+df['S_rot'].to_numpy()+df['S_trans'].to_numpy()

Gs = Hs - T*Ss
Gs

In [ ]:

df.insert(1,'H_533',Hs) #kcal/mol -> kJ/mol (occured at import)
df.insert(1,'S_533',Ss) #kcal/mol -> kJ/mol (occured at import)
df.insert(1,'G_533',Gs) #kcal/mol -> kJ/mol (occured at import)
df

In [ ]:
df.to_csv('gas_thermo_533.csv')

In [ ]:
kB = 1.380649E-23
h = 6.62607015E-34
T = 298

#rate constant per hour
k_rxn = kB*T/h*np.exp(-(19.5*4.1844)/8.314E-3/T)*3600
print(k_rxn)
# rate per hour
p_H2 = 1 #bar
rate = k_rxn*p_H2
#0.5 mM concentration, gives very low chemical activity of phenylacetylene
#barrier seems too high!
#TOF per hour r = 13756 hr-1
rate